# Clinical trials: ML Classification


In [1]:
import time
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
import numpy as np

In [2]:
base_color = "#3298D0"
plot_size = (14, 10)

sys.stdout = open('/dev/stdout', 'w')

In [3]:
# Path for csv folder & file
path_to_csv_file = os.path.abspath('../data/csv/')

# json file
csv_file = '/clean_data'

## Import csv file

In [4]:
# Import json into a dataframe
clean_csv_file = '{}{}.csv'.format(path_to_csv_file, csv_file)

In [5]:
'''Use dask to improve data loading
https://www.kaggle.com/shikhar1/yet-another-pandas-tutorial'''

# breaks with large json file
df = pd.read_csv(clean_csv_file)

In [6]:
df.shape

(10000, 10)

In [7]:
# remove Unnamed column
df = df.drop(columns = ['Unnamed: 0'])
df = df.drop(columns = ['original_date'])

In [8]:
df.columns

Index(['id', 'source', 'brief_title', 'condition', 'full_description',
       'summary', 'full_date', 'year'],
      dtype='object')

## Optimize memory usage

In [9]:
print(df.memory_usage(deep=True))
df.info(verbose=True)

In [10]:
# Change data types
df['id'] = df['id'].astype('str')
for col in ['source', 'condition']:
    df[col] = df[col].astype('category')

In [11]:
df['full_date'] =  pd.to_datetime(df['full_date'])

In [12]:
print(df.memory_usage(deep=True))
df.info(verbose=True)

In [13]:
print(df.shape)
print(df.nunique())

## List of conditions
- ftp://ftp.cdc.gov/pub/Health_Statistics/NCHS/Publications/ICD10CM/2019/

In [14]:
import xml.etree.ElementTree as ET

In [15]:
list_conditions = '../data/icd10/'

conditions = []

for xml in os.listdir(list_conditions):
    conditions.append(xml)

print(conditions)

In [16]:
icd = list_conditions + conditions[0] #full path
tree = ET.parse(icd).getroot()
print('Parsed: {}\n'.format(conditions[0]))

In [24]:
#Save titles from xml file in a list
all_conditions = []
    
for i in tree.iter('desc'):
    all_conditions.append(i.text)
    
len(all_conditions)

44803

In [25]:
# Remove long names
result = []

def count_words_list(words,size):
    for w in words:
        if len(w.split())<=size:
            result.append(w)
    return result

count_words_list(all_conditions, 3)

['Cholera',
 'Cholera, unspecified',
 'Typhoid fever',
 'Typhoid fever, unspecified',
 'Typhoid meningitis',
 'Typhoid pneumonia',
 'Typhoid arthritis',
 'Typhoid osteomyelitis',
 'Paratyphoid fever A',
 'Paratyphoid fever B',
 'Paratyphoid fever C',
 'Paratyphoid fever, unspecified',
 'Other salmonella infections',
 'Salmonella enteritis',
 'Salmonella sepsis',
 'Localized salmonella infections',
 'Salmonella meningitis',
 'Salmonella pneumonia',
 'Salmonella arthritis',
 'Salmonella osteomyelitis',
 'Salmonella pyelonephritis',
 'Salmonella infection, unspecified',
 'Shigellosis',
 'Other shigellosis',
 'Shigellosis, unspecified',
 'Campylobacter enteritis',
 'Foodborne staphylococcal intoxication',
 'Botulism food poisoning',
 'Amebiasis',
 'Acute amebic dysentery',
 'Chronic intestinal amebiasis',
 'Amebic nondysenteric colitis',
 'Ameboma of intestine',
 'Amebic liver abscess',
 'Amebic lung abscess',
 'Amebic brain abscess',
 'Cutaneous amebiasis',
 'Amebic cystitis',
 'Other ame

In [27]:
all_conditions = list(set(result))
len(all_conditions)

6088

## Classify docs using the list of conditions

In [18]:
'''
Text preparation
'''
# Create a column with all text
df['text'] = df['brief_title'] + df['summary'] + df['full_description']

In [19]:
# Clean extra blank spaces
df.text = df.text.replace('\s+', ' ', regex=True)

### Check for values in conditions list

In [23]:
# Create dataframe to store dictionary
dictionary_conditions = pd.DataFrame()

In [ ]:
for term in all_conditions:
    try:
        dictionary_conditions[term] = df['text'].str.contains(term, case = False).astype(int)
        print(term)
    except:
        pass

/Users/cmserna/miniconda3/envs/clinical-trials/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


## ML Pipeline

### DF transformations
- Check which ones can be added to the preprocessor
- Info: http://www.ultravioletanalytics.com/blog/tf-idf-basics-with-pandas-scikit-learn

In [ ]:
# Create a column combining title + full description
df['text'] = df['brief_title'] + df['summary'] + df['full_description']

In [ ]:
# Clean extra blank spaces
df.text = df.text.replace('\s+', ' ', regex=True)

### Create a vocabulary

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

In [ ]:
pat_numbers = lambda x: re.sub(r'(\d)+', '', x.lower())

cv = CountVectorizer(stop_words='english',
                     preprocessor = pat_numbers,
                     max_features = 3000,
                     lowercase = True,
                     max_df = 0.5,
                     ngram_range = (1, 3))

In [ ]:
docs = list(df['full_description'])

X = cv.fit_transform(docs)

In [ ]:
len(cv.vocabulary_)

In [ ]:
cv.vocabulary_

In [ ]:
# Check for values in dictionary
if 'breast cancer' in list(cv.vocabulary_):
  print('yes')
else:
  print('no')

In [ ]:
pd.DataFrame(X.toarray(), columns=cv.get_feature_names()).head()

### TF-IDF Model

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
transformer = TfidfTransformer()
tweights = transformer.fit_transform(X)
tweights

In [ ]:
# Pending: Reduced dimensionality



In [ ]:
# turn weights data into a dataframe
tf = pd.DataFrame(tweights.toarray(), columns=cv.get_feature_names())

In [ ]:
# Top terms by average tf-idf weight
weights = np.asarray(tweights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cv.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(5)

In [ ]:
# Check word
def check_weight(word):
    for w in word:
        try:
            print('{}: {}'.format(w, tf[w].mean()))
        except KeyError:
            print('{}: None'.format(w))

search_terms = ['cancer', 'breast cancer', 'migraine', 'sarcoma']
check_weight(search_terms)

In [ ]:
# Merge df y tf-idf data by index
pd.set_option('display.max_columns', None) 
tf.sample(5)
# pd.merge(df, tf, left_index=True, right_index=True).sample(1)

### Topic Modeling
- https://nlpforhackers.io/topic-modeling/

### Test: Using Tensorflow 
- https://github.com/open-source-for-science/TensorFlow-Course?utm_campaign=explore-email&utm_medium=email&utm_source=newsletter&utm_term=weekly